In [45]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
import utils
%matplotlib inline

In [46]:
data = loadmat("ex4data1.mat")
data.keys()
X,y=data['X'],data['y'].ravel()
y[y==10]=0
m=y.size
print(X.shape)
print(y.shape)
random_indices=np.random.choice(m,20,replace=False)
demo_pic=X[random_indices,:]
pyplot.imshow(demo_pic.reshape(-1,20).T)
pyplot.axis(False)

(5000, 400)
(5000,)


(-0.5, 399.5, 19.5, -0.5)

In [47]:
input_layer_size=400
hidden_layer_size=25
num_labels=10
weights=loadmat("ex4weights.mat")
theta1,theta2=weights['Theta1'],weights['Theta2']
theta2=np.roll(theta2,1,axis=0)
print(theta1.shape)
print(theta2.shape)
np_params=np.concatenate((theta1.ravel(),theta2.ravel()))
print(np_params.shape)

(25, 401)
(10, 26)
(10285,)


In [48]:
def sigmoid(z):
    sig_array=np.zeros(z.shape)
    sig_array=1.0/(1+np.exp(-z))
    return sig_array
def sigmoidGradient(z):
    sigg_array=np.zeros(z.shape)
    sigg_array=sigmoid(z)*(1-sigmoid(z))
    return sigg_array
"""
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:\n  ')
print(g)
"""
def costFunction(nn_params,input_layer_size,hidden_layer_size,num_labels,X,y,lambda_=0.0):
    theta1=np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)],(hidden_layer_size,(input_layer_size+1)))
    theta2=np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):],(num_labels,(hidden_layer_size+1)))
    m=y.size
    #print(theta1.shape)
    #print(theta2.shape)
    J=0
    theta1_grad=np.zeros(theta1.shape)
    theta2_grad=np.zeros(theta2.shape)
    a1=np.concatenate((np.ones((m,1)),X),axis=1)
    #print(a1.shape)
    a2=sigmoid(np.dot(a1,theta1.T))
    a2=np.concatenate((np.ones((a2.shape[0],1)),a2),axis=1)
    #print(a2.shape)
    a3=sigmoid(np.dot(a2,theta2.T))
    #print(a3.shape)
    #a3 is 5000* 10 matrix for each row(example) we have to multiply with actual res which will be k-dim array and only true class will contain value=1
    #np.eye(k)[label]  of k column, label selects which elements of that row==1 row number decided by no of elements in label
    y_matrix=y.reshape(-1)#vector of x, since we have done ravel for y, it is not required
    y_matrix=np.eye(num_labels)[y_matrix]
    temp1=theta1
    temp2=theta2
    cost_regterm=((lambda_)/(2*m))*(np.sum(np.square(temp1[:,1:]))+np.sum(np.square(temp2[:,1:])))
    #element wise multiplication. No transpose needed
    J=(-1/m)*np.sum((np.log(a3)*y_matrix)+np.log(1-a3)*(1-y_matrix))+ cost_regterm
    delta_3=a3-y_matrix
    #print(delta_3.shape)
    #we dont find delta2 w.rt to bias unit and hecne will start from column1
    delta_2=delta_3.dot(theta2)[:,1:]*sigmoidGradient(a1.dot(theta1.T))
    #print(delta_2.shape)
    Delta1=delta_2.T.dot(a1)
    Delta2=delta_3.T.dot(a2)
    theta1_grad=(1/m)*Delta1
    theta2_grad=(1/m)*Delta2
    theta1_grad[:,1:]=theta1_grad[:,1:]+(lambda_/m)*theta1[:,1:]
    theta2_grad[:,1:]=theta2_grad[:,1:]+(lambda_/m)*theta2[:,1:]
    grad=np.concatenate((theta1_grad.ravel(),theta2_grad.ravel()))
    return J,grad
lambda_=0
cost,grad=costFunction(np_params,input_layer_size,hidden_layer_size,num_labels,X,y,3)
print("cost : ",cost)
print(grad.shape)
print("gradient for thetas: ",grad)
#if Weights initialisation are very high then while calculation delta in backpropogation, multiplying will increase result exponentially or vice versa
#use Xavier initialisation mean of activation =0, variance of activation should stay almost same 
#def initialize_parameters_he(layers_dims):


cost :  0.5760512469501331
(10285,)
gradient for thetas:  [ 6.18712766e-05 -6.33744979e-12  1.31648811e-12 ...  3.66775518e-04
  2.91880885e-04  7.79237711e-05]


In [49]:
"""
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1]) * np.sqrt(2/layers_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1)) 
        ### END CODE HERE ###
        
    return parameters
    parameters = initialize_parameters_random([3, 2, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
    def randInitializeWeights(L_in,L_out,epsilon_init=0.12):
    W=np.zeros((L_out,1+L_in))
    W=np.random.rand(L_out,1+L_in)*2*epsilon_init-epsilon_init
    return W
print("Initializing theta parameters: ")
initial_theta1=randInitializeWeights(input_layer_size,hidden_layer_size)
initial_theta1=randInitializeWeights(hidden_layer_size,num_labels)
"""

#Initialization of parameters
#relu= (W[l],W[l-1])*sqrt(2/n[l-1])-relu xavier mein 1/n[l-1] or use 2/n[l-1]+n[L]
#basic using initialization in andrew ng

def randInitializeWeights(L_in,L_out,epsilon_init=0.12):
    W=np.zeros((L_out,L_in+1))
    #dimensions in random.randn
    W=np.random.randn(L_out,L_in+1)*2*epsilon_init-epsilon_init
    #W=np.random.randn(L_out,L_in+1)*np.sqrt(2/(L_out+L_in))
    return W
print('Initializing Neural Network Parameters ...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Unroll parameters
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)



Initializing Neural Network Parameters ...


In [50]:
#Gradient Checking
#again you have to implement it from starting
#left for future


In [51]:
#learning parameters with help of scipy.optimize
options={'maxiter':100}
lambda_=1
costFunc=lambda p: costFunction(p,input_layer_size,hidden_layer_size,num_labels,X,y,lambda_)
res=optimize.minimize(costFunc,initial_nn_params,jac=True,method='TNC',options=options)
nn_params=res.x
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))
print(Theta1)
print(Theta2)
def predict(theta1,theta2,X):
    m=X.shape[0]
    num_labels=theta2.shape[0]
    p=np.zeros((m,1))
    X=np.concatenate([np.ones((m,1)),X],axis=1)
    a2=sigmoid(np.dot(X,theta1.T))
    a2=np.concatenate((np.ones((a2.shape[0],1)),a2),axis=1)
    p=np.argmax(sigmoid(np.dot(a2,theta2.T)),axis=1)
    return p
pred=predict(Theta1,Theta2,X)
print("Training set accuracy for lambda=1: ",np.mean(pred==y)*100)

[[-0.38635475  0.12965898 -0.07614061 ... -0.2367378   0.29112024
   0.3726053 ]
 [-0.02619918 -0.09193459 -0.38453113 ... -0.06765183 -0.29500528
   0.01450307]
 [ 0.63968717  0.04894513 -0.11311462 ...  0.09753409  0.20034351
  -0.25408874]
 ...
 [ 0.26749804  0.08209611  0.05643026 ... -0.09849583  0.21491489
  -0.04013558]
 [-0.31433061 -0.21932181 -0.15382136 ... -0.21822292 -0.13626195
  -0.15002946]
 [-0.00270937 -0.05797095 -0.29005441 ... -0.39164038 -0.29712126
  -0.00540201]]
[[ 2.78202695e+00 -3.27935122e-01 -5.79795789e-01 -2.91371578e+00
   1.44846067e+00 -4.10599581e-02 -4.91727169e-01 -5.17139041e-01
  -3.77313141e-01  5.52804008e-02 -1.60094143e-01 -1.01047032e-01
  -4.52406888e-01  1.87520556e-02 -5.52284077e-01 -6.60786225e+00
  -2.87799805e-01 -6.04260572e-02 -3.20580044e+00 -9.95067284e-01
  -2.34778469e+00 -4.15615409e+00 -6.20375780e-01 -9.21406689e-01
  -4.01463897e+00 -2.62359786e-01]
 [-1.19857595e+01  5.56491764e-01  2.47167578e+00 -5.45036937e-01
   7.633033